In [1]:
#importing modules and matplotlib PyQt5 backend
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

In [2]:
#BIDS: Brain Imaging Data Structure to organise EEG datasets
#specifying input folder (sample bids data)
bids_root = pathlib.Path('out_data/sample_BIDS')

#bids info
bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)
#reading the input
raw = mne_bids.read_raw_bids(bids_path)
#loading the data
raw.load_data()
#filtering
raw.filter(l_freq=0.1, h_freq=40)
#extracting events
events, event_id = mne.events_from_annotations(raw)

Opening raw data file out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading events from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv.
Reading channel info from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


C:\Users\Dell\AppData\Local\Temp\ipykernel_11392\4265811384.py:13: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 19821 samples (33.001 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Left', 'Visual/Right']


[Parallel(n_jobs=1)]: Done 366 out of 366 | elapsed:    9.2s finished


In [3]:
#experimental events are found like stimulus, trigger channels, etc
#events are encoded with event ids which are not readable by humans
#therefore we create a dictionary to map these event ids to readable forms
#events are needed to make epochs
event_id

{'Auditory/Left': 1,
 'Auditory/Right': 2,
 'Button': 3,
 'Smiley': 4,
 'Visual/Left': 5,
 'Visual/Right': 6}

In [4]:
#where epoch starts (before the event)
tmin = -0.3
#where epoch ends (after the event)
tmax = 0.5
baseline = (None, 0)
#epochs slice the continuous data into small time segments
#creating epochs
epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True)
epochs

Not setting metadata
320 matching events found
Setting baseline interval to [-0.2996928197375818, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 320 events and 481 original time points ...
0 bad epochs dropped


Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [5]:
#plotting EEG epochs
epochs.plot(picks='eeg')

Using matplotlib as 2D backend.


<MNEBrowseFigure size 800x749 with 4 Axes>

In [6]:
#saving the data
epochs.save(pathlib.Path('out_data') / 'epochs_epo.fif', 
            overwrite=True)

Overwriting existing file.


In [7]:
#taking auditory and visual event avg and feeding to a variable
evoked_auditory = epochs['Auditory'].average()
evoked_visual = epochs['Visual'].average()

In [8]:
#joint plot for auditory of eeg signals in the time domain
evoked_auditory.plot_joint(picks='eeg')

Projections have already been applied. Setting proj attribute to True.
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
['MEG 2443', 'EEG 053']


<Figure size 800x420 with 7 Axes>